In [11]:
from DataAcq.DataAcquisition import DataAcquisition
from DataNorm.DataNormalization import DataNormalization
from Plot.DataPlotter import DataPlotter
from DataClean.DataCleaning import DataCleaning
from model.model import CNNClassifier



In [7]:
# Usage Example:
try:
    data = loaded_data.iloc[24000000:]
    labels=data.iloc[:,0]
    data=data.iloc[:,1:5]    
    normalized_data = DataNormalization(method='MSD').fit_transform(data)
except ValueError as ve:
    print(ve)
except Exception as e:
    print("An error occurred:", e)

print(normalized_data)
print (data)
labels=labels.to_numpy()
print(labels)


[[-5.22150921e-01 -1.14030188e+00  2.00553010e+00 -4.55899791e-01]
 [-5.22150921e-01  9.37447279e-01 -6.22005358e-01 -1.24596075e+00]
 [-5.22150921e-01  1.59165600e+00 -1.42796510e+00 -1.70427228e+00]
 ...
 [-7.65628292e+00  1.80343881e-02  1.08496638e-02  6.51327647e-02]
 [-7.65628292e+00  1.87781824e-02  3.13858381e-03  1.65239590e-02]
 [-7.65628292e+00  2.41469961e-03  1.00603410e-02 -4.53238059e-03]]
          Measured_RPM  Vibration_1  Vibration_2  Vibration_3
24000000    2059.02550    -0.015743     0.040058     0.000284
24000001    2059.02550     0.014228    -0.011530    -0.003921
24000002    2059.02550     0.023664    -0.027354    -0.006360
24000003    2059.02550    -0.039741     0.046192     0.006880
24000004    2059.02550     0.005262    -0.007758     0.000144
...                ...          ...          ...          ...
26423290     643.38338     0.001339     0.000769     0.003015
26423291     643.38338     0.001261     0.000952     0.003138
26423292     643.38338     0.00096

In [8]:
labels.shape

(2423295,)

In [4]:
# Définir le chemin de stockage pour les données
file_path = "C:\\Users\\USER\\OneDrive\\Bureau\\github\\vibro\\0D.csv"

loaded_data = DataAcquisition.load_data(file_path)

In [7]:
data2=data.iloc[40000:]

In [ ]:
plotter = DataPlotter(data2, 100)
plotter.plot_series("Measured_RPM", percentage=100,title="Measured_RPM")
# plotter.plot_series("Vibration_2", percentage=100)
# plotter.plot_series("Vibration_3", percentage=100)

In [9]:
normalized_data.shape

(2423295, 4)

In [ ]:
plotter = DataPlotter(normalized_data, 100)
plotter.plot_series(2, percentage=100,title="normalized_data")
plotter.plot_series(3, percentage=100)
plotter.plot_series(4, percentage=100)

In [5]:
# Exemple d'utilisation
raw_data = normalized_data  # Votre jeu de données brut
cleaner = DataCleaning(raw_data)
cleaned_data = cleaner.run()

In [ ]:
plotter = DataPlotter(cleaned_data, 1)
plotter.plot_series(0, percentage=10,title="normalized_data")
# plotter.plot_series(3, percentage=100)
# plotter.plot_series(4, percentage=100)

In [13]:
seq_length = 10000  # Longueur de la séquence
features = data.shape[1]  # Nombre de caractéristiques
print(features)
def create_sequences(data, seq_length):
    sequences = []
    for i in range(len(data) - seq_length):
        seq = data[i:i + seq_length]
        sequences.append(seq)
    return np.array(sequences)

sequences = create_sequences(data, seq_length)

4


In [6]:
data_tensor = torch.Tensor(sequences)
labels_tensor = torch.Tensor(labels).long()

dataset = TensorDataset(data_tensor, labels_tensor)
dataloader = DataLoader(dataset, batch_size=1000, shuffle=True)

In [8]:
hidden_size = 128  # Taille de l'état caché
num_layers = 3     # Nombre de couches LSTM
input_size= 4
model = LSTMModel(input_size, hidden_size, num_layers,1)


In [9]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)


In [13]:
X_train, X_test, y_train, y_test = train_test_split(data, labels, test_size=0.2, random_state=42)

input_shape = X_train.shape[1:]
num_classes = np.unique(y).shape[0]

classifier = CNNClassifier(input_shape=input_shape, num_classes=num_classes)
classifier.train(X_train, y_train, X_test, y_test)
score = classifier.evaluate(X_test, y_test)
print(f'Test loss: {score[0]}')
print(f'Test accuracy: {score[1]}')

# Saving and loading the model
model_filepath = 'best_model.h5'
classifier.train(X_train, y_train, X_test, y_test, model_filepath=model_filepath)
classifier.load(model_filepath)
# Further evaluation or inference can be done here

NameError: name 'train_test_split' is not defined

Note: you may need to restart the kernel to use updated packages.


ERROR: Could not find a version that satisfies the requirement tensflow (from versions: none)
ERROR: No matching distribution found for tensflow

[notice] A new release of pip is available: 23.1.2 -> 23.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip
